In [2]:
import pandas as pd 
import pymysql 
import pymysql.cursors
import numpy as np

df = pd.DataFrame()
df_update = {}
insert_column = ()
update_column = [] 

ssh_host = '3.36.21.112'
ssh_port = '21'
conn_resource = pymysql.connect(host='3.36.21.112', user='root', password='ckUdC17yqltr', db='resource')
conn_testserver = pymysql.connect(host='3.36.21.112', user='root', password='ckUdC17yqltr', db='saladpet')

cursor_resource = conn_resource.cursor() #dictionary 형태로 결과 반환 원해 -> pymysql.cursors.Dictcursor을 ()에 넣어줌.
cursor_testserver = conn_testserver.cursor()

## df_to_sql_resource 함수 : dataframe 형태를 sql에 upsert 할 수 있게 해줌 (단, 칼럼명이 sql의 칼럼명과 동일해야 함!)
def df_to_sql_resource (df, table_name, primary_key) : # df = 넣을 dataframe, table_name = sql 테이블 이름, column = df와 동일한 순서의 칼럼, primary_key = PK or unique key로 on duplicate key 시에 사용하지 않는 것
    column = df.columns.values.tolist()
    insert_column = tuple(column)
    update_column = column 
    for i in range (0,len(primary_key)):
        update_column.remove(primary_key[i])
    update_column = tuple(update_column)

    word_1 = "insert into %s (" %table_name 
    word_2 = "on duplicate key update" 

    for l in range(0,len(update_column)):
        if l < len(update_column)-1 :
            word_2 = word_2 + " "+ update_column[l] + " = %s,"
        if l == len(update_column)-1 :
            word_2 = word_2 + " " + update_column[l] + " = %s"

    for s in range(0,len(insert_column)):
        if s < len(insert_column)-1 :
            word_1 = word_1 + " "+ insert_column[s] + ","
        if s == len(insert_column)-1 :
            word_1 = word_1 + " "+ insert_column[s] + " ) "
    word_1 = word_1 + "VALUES (" + "%s,"*(len(insert_column)-1) + "%s)"
    print (word_1 + word_2)

    for i in range (0,len(df)) :
        a = df.iloc[i] # index = i 인 df를 series로 변환 
        insert_data = a.to_dict() # series를 사전 형태로 변환 - insert 사전 목록 
        update_data = a.to_dict() 
        for key in primary_key:
            if key in update_data:
                del update_data[key] # series를 사전 형태로 변환 - update 사전 목록 
        data = list(insert_data.values()) + list(update_data.values()) #insert+update 목록 리스트 
        sql = word_1 + word_2
        cursor_resource.execute(sql,data)
        conn_resource.commit()
        i = i+1
    print ("finish insert %s" %table_name )
    print ("넣은 row 수 총 %s 개" %i ) 
    conn_resource.close()


def df_to_sql_testserver (df, table_name, primary_key) : # df = 넣을 dataframe, table_name = sql 테이블 이름, column = df와 동일한 순서의 칼럼, primary_key = PK or unique key로 on duplicate key 시에 사용하지 않는 것
    column = df.columns.values.tolist()
    insert_column = tuple(column)
    update_column = column 
    for i in range (0,len(primary_key)):
        update_column.remove(primary_key[i])
    update_column = tuple(update_column)

    word_1 = "insert into %s (" %table_name 
    word_2 = "on duplicate key update" 

    for l in range(0,len(update_column)):
        if l < len(update_column)-1 :
            word_2 = word_2 + " "+ update_column[l] + " = %s,"
        if l == len(update_column)-1 :
            word_2 = word_2 + " " + update_column[l] + " = %s"

    for s in range(0,len(insert_column)):
        if s < len(insert_column)-1 :
            word_1 = word_1 + " "+ insert_column[s] + ","
        if s == len(insert_column)-1 :
            word_1 = word_1 + " "+ insert_column[s] + " ) "
    word_1 = word_1 + "VALUES (" + "%s,"*(len(insert_column)-1) + "%s)"
    print (word_1 + word_2)

    for i in range (0,len(df)) :
        a = df.iloc[i] # index = i 인 df를 series로 변환 
        insert_data = a.to_dict() # series를 사전 형태로 변환 - insert 사전 목록 
        update_data = a.to_dict() 
        for key in primary_key:
            if key in update_data:
                del update_data[key] # series를 사전 형태로 변환 - update 사전 목록 
        data = list(insert_data.values()) + list(update_data.values()) #insert+update 목록 리스트 
        sql = word_1 + word_2
        cursor_testserver.execute(sql,data)
        conn_testserver.commit()
        i = i+1
    print ("finish insert %s" %table_name )
    print ("넣은 row 수 총 %s 개" %i ) 
    conn_testserver.close()


In [71]:
################# 1. 필요한 pandas, pymysql 셋팅 작업 #################
import pandas as pd 
import pymysql 
import pymysql.cursors


#######TEST_SERVER
ssh_host = '3.36.21.112'
ssh_port = '22'
conn_resource = pymysql.connect(host='3.36.21.112', user='root', password='ckUdC17yqltr', db='resource')
conn_testserver = pymysql.connect(host='3.36.21.112', user='root', password='ckUdC17yqltr', db='saladpet')


#######SERVER
# ssh_host = '15.165.92.95'
# ssh_port = '12200'
# conn_resource = pymysql.connect(host='15.165.92.95', user='root', password='COLCTveCNfY8', db= 'resource')
# conn_testserver = pymysql.connect(host='15.165.92.95', user='root', password='COLCTveCNfY8', db='saladpet')

#################################################################################################
################## 2. 엑셀 전처리 시 필요한 SQL DB 불러오기 작업 ##################

cursor_resource = conn_resource.cursor() #dictionary 형태로 결과 반환 원해 -> pymysql.cursors.Dictcursor을 ()에 넣어줌.
sql_supplement_type = "select * from supplement_type"
sql_map_product_cate = "select * from map_product_cate"

supplement_type = pd.read_sql(sql_supplement_type, conn_resource) # 열이 포함된 형태로 select하려면 이 문자형태로 진행 
map_product_cate = pd.read_sql(sql_map_product_cate, conn_resource)
conn_resource.commit

#supplement_type.set_index("type_id", inplace = True ) # inplace = true는 변경 사항을 그대로 객체에 다시 저장한다.
#map_product_cate.set_index("mc_pk", inplace = True) 



################## 3. 엑셀 불러오기 작업 ####################
excel_path = "0906_supplement.xlsx"
df_pro = pd.read_excel( excel_path , sheet_name = "product",usecols= [0,1,2,3,4,5,6,7,8,9,10,11,12]) 
df_nu = pd.read_excel( excel_path , sheet_name = "nutrient",usecols = [0,1,2,3,4,5,6,7])
df_cal = pd.read_excel(excel_path, sheet_name = "calorie")
#df_fun = pd.read_excel(excel_path, sheet_name = "function_ingredient")
df_price = pd.read_excel( excel_path, sheet_name = "price", usecols = [0,1,2,3,4,5,6])
df_ingredient = pd.read_excel( excel_path, sheet_name = "ingredient")

df_price['gift_unit'].astype(int)



################ 4. 데이터 전처리 작업 ####################


####### 문자를 숫자로 1.subtype_nm >> subtype_id 2. category(mc_nm) >> mc_pk
df_pro = pd.merge(df_pro, supplement_type, how = 'left', left_on = 'subtype_nm', right_on = 'type_nm')
df_pro = pd.merge(df_pro, map_product_cate, how = 'left', left_on = 'category', right_on = 'mc_nm')
df_pro["lc_pk"] = 3
#product = df_pro.iloc[:,[0,1,2,5,6,7,8,9]]

product = df_pro.loc [ : , ('product', 'prod_nm', 'brand_pk', 'salary_age', 'salary_type', 'pet_type', 'exp_type', 'exp_month') ]
#supplement_unit = df_pro.iloc[:,[0,8,9,10,11]]

supplement_unit = df_pro.loc [ : , ('product', 'least_unit', 'least_weight_unit', 'weight_unit', 'subtype_id')]
product_cate = df_pro.loc[ :, ('product', 'lc_pk', 'mc_pk')]
#product_cate = df_pro.iloc[:,[0,13,15]]



######## nan >> none(null)

# product = product.where(product.notnull(),None)
# supplement_unit = supplement_unit.where(supplement_unit.notnull(),None)
# df_price = df_price.where(df_price.notnull(),None)
# df_nu = df_nu.where(df_nu.notnull(),None)
# df_cal = df_cal.where(df_cal.notnull(),None)

#---------------------------------------------

#df_fun = df_fun.replace({np.nan:None})
supplement_unit = supplement_unit.replace({np.nan: None})
df_price = df_price.replace({np.nan: None})
df_nu = df_nu.replace({np.nan: None})
df_cal = df_cal.replace({np.nan: None})
product = product.replace({np.nan: None})
df_ingredient = df_ingredient.replace({np.nan: None})



##################### 마지막 확인#################
# product.info()
# supplement_unit.info()
# product_cate.info()
# df_price.info()
# df_nu.info()
# df_cal.info()
df_ingredient.head()

#product.head()
# supplement_unit.head()
# product_cate.head()
# df_price.head()
# df_nu.head()
# df_cal.head()
# df_ingredient.head()

#product.to_csv("products_mepet.csv", index=False)
#product.to_csv("products_mepet.csv", index=False)
# supplement_unit.to_csv("supplement_unit_mepet.csv", index=False)
# product_cate.to_csv("product_cate_mepet.csv", index=False)
# df_price.to_csv("search_price_detail_mepet.csv", index=False)
# df_nu.to_csv("product_nutrient_mepet.csv", index=False)
# df_cal.to_csv("product_calorie_mepet.csv", index=False)
# df_ingredient.to_csv("ingredient_mepet.csv", index=False)


##################### 집어넣기 (supplement_plan 제외)#############
# p_table_name = "products" 
# p_pk = ["product"] 

# unit_table_name = "supplement_unit"
# unit_pk = ["product"]

# cate_table_name = "product_cate"
# cate_pk = ["product"]


# price_table_name = "search_price_detail"
# price_pk = ["product"]

# nu_table_name = "product_nutrient"
# nu_pk = ["product"]

# cal_table_name = "product_calorie"
# cal_pk = ["product"]

# ing_table_name = "ingredient"
# ing_pk = ["product", "ingred_row"]

# df_to_sql_resource(product, p_table_name , p_pk )
# df_to_sql_resource(supplement_unit, unit_table_name , unit_pk)
# df_to_sql_resource(product_cate, cate_table_name, cate_pk )
# df_to_sql_resource(df_price, price_table_name, price_pk )
# df_to_sql_resource(df_nu, nu_table_name, nu_pk)
# df_to_sql_resource(df_cal, cal_table_name, cal_pk )
# df_to_sql_resource(df_ingredient, ing_table_name, ing_pk)

,product,ingred_row,ingred_name,function_ingredient_id,ingred_index
0,13087,1,플루란,None,15115
1,13087,2,스피루리나,None,3585
2,13087,3,글리세린,None,792
3,13087,4,합성착향료(블루베리향),None,15117
4,13087,5,프로폴리스추출물,None,11798


In [63]:
############################################### 새로운 시도###########################
import pymysql
import paramiko
import pandas as pd
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder
from os.path import expanduser

home = expanduser('~')
mypkey = paramiko.RSAKey.from_private_key_file("C:\Users\지상민\Downloads\LightsailDefaultKey-ap-northeast-2 (4).pem")
# if you want to use ssh password use - ssh_password='your ssh password', bellow

sql_hostname = 'bitnami'
sql_username = 'root'
sql_password = 'COLCTveCNfY8'
sql_main_database = 'db_name'
sql_port = 3306
ssh_host = 'ssh_hostname'
ssh_user = 'ssh_username'
ssh_port = 22
sql_ip = '1.1.1.1.1'

with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(sql_hostname, sql_port)) as tunnel:
    conn = pymysql.connect(host='127.0.0.1', user=sql_username,
            passwd=sql_password, db=sql_main_database,
            port=tunnel.local_bind_port)
    query = '''SELECT VERSION();'''
    data = pd.read_sql_query(query, conn)
    conn.close()

SyntaxError: invalid syntax (Temp/ipykernel_17720/1511572921.py, line 7)

In [69]:
### supplement plan 작업 
import pandas as pd
from numpy import nan as NA

excel_path = "0906_supplement.xlsx"

df_plan = pd.read_excel( excel_path, sheet_name = "supplement_plan")

df_plan = df_plan.replace({np.nan: None})
#df_plan = df_plan.where(df_plan.notnull(),None)  ## sql 에서는 모든 결측치 종류 중 none 만 null 로 판단함

cond_1 = (df_plan['attribute'].isnull())

df_plan.loc[cond_1, "attribute" ]  =  "일반적인 경우"

cond_2 = (df_plan['pet_weight_min'] == 0)   

df_plan.loc[ (df_plan['weight_unit'].isnull()) , 'pet_weight_min' ] = 0 
df_plan.loc[ (df_plan['weight_unit'].isnull()) , 'weight_unit' ] = "kg"
df_plan.loc[ (df_plan['pet_weight_min'].isnull()), 'pet_weight_min'] = 0  # 시작이 null인 데이터는, 0으로 셋팅해줘야 함 


df_plan_group = df_plan.groupby(['product','attribute','pet_type']) # 연속 데이터 처리 작업 1
index = df_plan_group.groups
#{(12841, '일반적인 경우', 1): [4, 5, 6], (12842, '일반적인 경우', 1): [7, 8, 9]} 이런식으로 되어있음 (dict 형태로)


for i in range (0, len(index)): 

    pk = list(index.values())[i] # index를 담는 list = pk로 정의, index.values() 는 dict 형태에서 : 오른쪽 꺼를 의미
    df_group = (df_plan.iloc[pk, : ]) # df_plan 중 해당 index를 가진 row를 df화 시킴 
    df_group = df_group.to_numpy().tolist() # df를 list화 시킴 

    #지금 df_group은 아래 두줄과 같이 되어있음
    #[[12841, '일반적인 경우', 1, 0.0, 5.0, 'kg', 1, 1.0, 1.0, 1, nan], [12841, '일반적인 경우', 1, 5.0, 10.0, 'kg', 1, 2.0, 2.0, 1, nan], [12841, '일반적인 경우', 1, 10.0, nan, 'kg', 1, 3.0, 3.0, 1, nan]]
    #[[12842, '일반적인 경우', 1, 0.0, 5.0, 'kg', 1, 1.0, 1.0, 1, nan], [12842, '일반적인 경우', 1, 5.0, 10.0, 'kg', 1, 2.0, 2.0, 1, nan], [12842, '일반적인 경우', 1, 10.0, nan, 'kg', 1, 3.0, 3.0, 1, nan]]
    
    
    if i == 0 :
        groups = [df_group]

    else :

        groups += [df_group] # group box에 추가함 
        groups

result = []     # 추가해야할 데이터 리스트

for group in groups:
    init_min, init_max = group[0][3], group[0][4]                    # 그룹 첫번째 row의 min, max 값
    prev_min, prev_max = init_min, init_max                          # 이전 min, max 값을 저장하는 변수 초기화

    if len(group) >= 2:     # 그룹에 row가 2개 이상 일 때
        
        for i, row in enumerate(group): # 1. 중간 데이터들에 대해서 연속성 추가 작업// i(앞에꺼)는 인덱스, ROW(뒤에꺼) 는 진짜 값
            if i == 0:                                               # row의 처음은 건너 뛰고 2번째 부터 비교
                continue
            cur_min, cur_max = row[3], row[4]                        # 현재 row의 min, max 값
            if prev_max != cur_min:                                  # 이전 최대값과 현재 최소값이 같지 않을 경우
                result.append( [group[i][0],group[i][1], group[i][2], prev_max, cur_min, group[i][5], group[i][6], group[i][7], group[i][8], group[i][9]] )
            prev_min, prev_max = cur_min, cur_max                    # row를 비교 한 뒤 값 최신화

        if group[0][3] != 0 :  # 2. 시작 데이터에 대해서 min이 0이 아닌 경우 추가 작업 
            result.append([group[0][0], group[0][1], group[0][2], 0, init_min , group[0][5], group[0][6], group[0][7], group[0][8], group[0][9]])

        if group[-1][4] is not None:      # 그룹의 마지막 row max 값이 Null 이 아닐 경우
            result.append([group[-1][0], group[-1][1], group[-1][2], group[-1][4], None, group[-1][5], group[-1][6], group[-1][7], group[-1][8], group[-1][9] ])        # 최대값이 NULL 인 데이터 추가
            
    else:       # 그룹에 row가 1개 일 때 (그룹에 row가 무조건 1개 이상 있다는 가정 하에)
        if group[0][3] != 0 and group[0][4] != None : 
            result.append([group[0][0], group[0][1], group[0][2], init_max, None, group[0][5], group[0][6], group[0][7], group[0][8], group[0][9]]) 
            result.append([group[0][0], group[0][1], group[0][2], 0, init_min , group[0][5], group[0][6], group[0][7], group[0][8], group[0][9]])
        if group[0][3] == 0 and group[0][4] != None  : 
            result.append([group[0][0], group[0][1], group[0][2], init_max, None, group[0][5], group[0][6], group[0][7], group[0][8], group[0][9]])                 # 최대값이 NULL 인 데이터 추가
            
        if group[0][3] != 0 and group[0][4] == None  :
            result.append([group[0][0], group[0][1], group[0][2], 0, init_min , group[0][5], group[0][6], group[0][7], group[0][8], group[0][9]]) # 최소값이 0이고, 최대가 init인 데이터 추가 

 
column = ['product', 'attribute', 'pet_type', 'pet_weight_min', 'pet_weight_max','weight_unit', 'time_per_day', 'min_amount', 'max_amount', 'per_unit']

df_plus = pd.DataFrame.from_records( result , columns = column )

def func1(row):
    if '~' in str(row) :
        row_1 = row.split('~')[0]
        row_2 = row.split('~')[1]
    else :
        row_1 = row
        row_2 = row 
    return pd.Series([row_1, row_2])

df_plan = pd.concat([df_plus, df_plan])

df_plan[['time_per_day_min', 'time_per_day_max']] = df_plan['time_per_day'].apply(func1)

df_plan['time_per_day_min']= df_plan['time_per_day_min'].astype(int)
df_plan['time_per_day_max']= df_plan['time_per_day_max'].astype(int)
df_plan = df_plan.drop(columns = 'time_per_day')

con_3 = ( df_plan['pet_weight_max'].isnull() )

df_plan.loc[con_3, 'pet_weight_max']  = 1000 


df_plan.loc[ (df_plan['pet_weight_min'].isnull()), 'pet_weight_min' ] = 0


##################### 확인 (0~1000 잘 되어있는지)
df_plan.head(10)
# <개수 확인> min의 0 개수 = max의 1000 개수 = product, attribute, pet_type 을 pk 로 잡았을때 중복제거 한 값 개수

df_plan.to_csv("supplement_plan_mepet.csv", index=False)
### 집어넣기

#df_plan_table = "supplement_plan"
#df_plan_pk = ["product", "attribute", "pet_type", "pet_weight_min", "pet_weight_max"]
#df_to_sql_resource (df_plan, df_plan_table, df_plan_pk )


